### 01 — Exploratory Data Analysis (EDA)

**Local data source:** `/data/interim/10kDiabetes.csv`

## Executive Summary

This notebook performs an initial sweep of the Diabetes Readmission dataset.

**Findings:**
- The dataset is highly imbalanced, with far fewer readmissions than non-readmissions.
  This makes ROC-AUC less informative and increases the importance of PR-AUC.

- Many features are categorical (e.g., admission source/type, discharge disposition, race, gender),
  requiring one-hot encoding for classical ML models.

- Several numerical features (e.g., number of inpatient visits, emergency visits, diagnoses)
  show long-tailed distributions and strong predictive potential.

- ICD-9 diagnostic codes are messy and high-cardinality; each patient has three diagnosis fields,
  and codes range from broad categories to highly granular subcodes.  
  This leads to a very sparse and wide feature space if one-hot encoded directly.
  I explore this more in [[03_ICD9_feature_engineering_prototype](./03_ICD9_feature_engineering_prototype.ipynb)] and [[04_Model-v2.ipynb](./04_Model-v2.ipynb)].  

- Missingness varies substantially by field; some categorical features (e.g., medical specialty)
  contain large proportions of ‘Unknown’ or ‘Missing’ values.

- Preliminary relationships (e.g., higher inpatient/emergency visit counts) suggest clinically
  plausible drivers of readmission that will be explored further in [[02_Model.ipynb](./02_Model.ipynb)].

#### 1. Imports and Project Setup

In [ ]:
import sys
import os

project_root = os.path.abspath("..") # Add project root to Python path so we can import data_loader etc etc
if project_root not in sys.path:
    sys.path.append(project_root)

project_root

In [ ]:
# Minimal utility for loading the diabetes readmission dataset from data/interim/.

import pandas as pd

INTERIM_DIR = os.path.join("data", "interim")
DIABETES_DATA_CSV = "10kDiabetes.csv"

def load_diabetes_csv(filename, project_root=''):
    csv_path = os.path.join(project_root, INTERIM_DIR, filename)

    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSV file not found: {csv_path}")

    return pd.read_csv(csv_path)

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore") # Not too bothered about message "Pyarrow will become a required dependency of pandas"
    import pandas as pd
    from IPython.core.display import display, HTML

import numpy as np
import missingno as msno
import re
import itertools

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid", font_scale=1.4)  # try 1.3–1.6

plt.style.use("seaborn-v0_8")
pd.set_option("display.max_columns", None)

In [ ]:
url = "https://raw.githubusercontent.com/clarkian-teachings/clarkian-python-ml-intro/main/data/10kDiabetes.csv"
df = pd.read_csv(url)
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

This summary shows:
- column data types (yes there are 50 columns, plus a numerical index!)
- missingness structure  
- categorical vs numerical balance

## Which fields have missing values/NaNs?

In [ ]:
missing = df.isna().sum().sort_values(ascending=False)
missing[missing > 0]
missing_df = (
    missing.to_frame(name="num_missing")
        .assign(
            pct_missing=lambda x: (x["num_missing"] / len(df) * 100).round(2),
            pct_present=lambda x: 100 - x["pct_missing"]
        )
)
print(f"\nNumber of rows: {df.shape[0]}")
missing_df

## Heatmap of missing values

In [ ]:
plt.figure(figsize=(16, 16))

ax = msno.matrix(  # Missingness matrix
    df,
    labels=True,
    fontsize=14,
)

fig = plt.gcf()
fig.canvas.draw()

for artist in fig.findobj(match=plt.Text):
    if artist.get_text() == "Data Completeness":
        artist.set_rotation(90)
        artist.set_fontsize(12)
        artist.set_horizontalalignment('left')
        artist.set_verticalalignment('bottom')

plt.xticks(rotation=90, fontsize=12)
plt.title("Missing Values Heatmap", fontsize=16)
plt.subplots_adjust(bottom=0.35, top=0.90)
plt.show()

Data completeness is actually pretty good, with the notable exception of fields **max_glu_serum** and **A1Cresult**.

There appears to be no particular clustering when it comes to missing data attributes.

Given the low overall missingness, I am pretty confident in using simple imputation -- nothing complex needed here!

## Target Distribution (Binary Readmission Flag)

Check the Boolean variable we're trying to predict for how well it is balanced across the dataset

In [ ]:
target_col = "readmitted"
pct = df[target_col].value_counts(normalize=True).mul(100).round(2)
pct_df = pct.rename("pct").astype(str) + '%'
pct_df.to_frame()

N.B. 60/40 is not too much of an imbalance. But we will go carefully, being mindful of Type I/Type II statistical errors.

**We should be particularly cautious about false negatives**, which have both clinical and operational implications: if a patient is at risk of readmission, we don’t want the model to miss them - we have a **duty of care** to patients.

## Investigate numerical features

In [ ]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
if "rowID" in numeric_cols:
    numeric_cols.remove("rowID")

df_numeric = df[numeric_cols]
df_numeric.describe().T

No missing values. I can use this.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 6))
sns.violinplot(data=df_numeric, inner='quartile', cut=0)
plt.title("Violin plots for numerical features")
plt.xticks(rotation=45)
plt.show()

### Let's see the distributions separately for (i) patients who were readmitted, and (b) patients who weren't.

In [ ]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != "rowID"]

df_long = df.melt(
    id_vars="readmitted",
    value_vars=numeric_cols,
    var_name="feature",
    value_name="value"
)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

numeric_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != "rowID"]

scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[numeric_cols] = scaler.fit_transform(df[numeric_cols])

df_long_scaled = df_scaled.melt(
    id_vars="readmitted",
    value_vars=numeric_cols,
    var_name="feature",
    value_name="value"
)

sns.set_theme(style="whitegrid", font_scale=1.5)
plt.figure(figsize=(20, 12))

sns.violinplot(
    data=df_long,
    x="feature",
    y="value",
    hue="readmitted",
    split=True,
    inner=None,
    cut=0,
    width=0.9,        # <-- MUCH WIDER VIOLINS
    density_norm="count",    # <-- Uses sample size to expand violins
    bw_adjust=1.2,    # <-- optional smoothing
    linewidth=1.0,
)

plt.title("Half-Violin Comparison (Standardised Features)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Patients who are readmitted tend to have more lab procedures, medication counts skewed slightly highly, and slightly longer hospital stays.

This early stage view suggests that the greater interaction with hospital procedures, _possibly_ the higher the risk of readmission.

In [ ]:
cols = ['readmitted', 'time_in_hospital', 'num_lab_procedures', 'num_medications']
df_sub = df[cols]

summary = (
    df_sub
    .groupby('readmitted')
    .agg(['mean', 'median', 'std', 'min', 'max']).T
)
summary

In [ ]:
summary2 = (
    df_sub
    .groupby('readmitted')
    .agg(['mean', 'median', 'std']).T
)
summary2.columns = ['Not_readmitted', 'Readmitted']
summary2['% Difference'] = (
    (summary2["Readmitted"] / summary2["Not_readmitted"] - 1) * 100
).round(1)

summary2['% Difference'] = summary2['% Difference'].astype(str) + '%' # Convert to string with % symbol
summary2

## Number of Inpatient Visits

In [ ]:
sns.histplot(df['number_inpatient'], bins=20)
plt.title("Distribution of Number of Inpatient Visits")
plt.show()

Most patients have zero or one inpatient visit, while a smaller minority have repeated admissions, as one would expect.

### Note that there appears to be some dependency structure between this and hospital readmission.

In [ ]:
sns.histplot(
    data=df,
    x='number_inpatient',
    hue='readmitted',
    bins=20,
    stat='count',
    common_norm=False,   # prevents normalization across groups
    element='step'       # optional: cleaner overlapping lines
)

plt.title("Number of Inpatient Visits by Readmission Status")
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

sns.histplot(
    df[df['readmitted'] == 0]['number_inpatient'],
    bins=20,
    ax=axes[0],
    stat='density',
    color='steelblue'
)
axes[0].set_title("Not Readmitted")
axes[0].set_xlabel("number_inpatient")

sns.histplot(
    df[df['readmitted'] == 1]['number_inpatient'],
    bins=20,
    ax=axes[1],
    stat='density',
    color='darkorange'
)
axes[1].set_title("Readmitted")
axes[1].set_xlabel("number_inpatient")

plt.suptitle("Distribution of Inpatient Visits by Readmission Outcome")
plt.tight_layout()
plt.show()

While most patients have zero or one inpatient visit, the readmitted group shows a visibly higher frequency of repeated visits (≥2).

Repeated prior inpatient admittance appears predictive of readmission risk.

## Distribution of Age

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

age_order = [ '[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)', '[60-70)', '[70-80)', '[80-90)', '[90-100)' ]

clean_labels = ["0–10", "10–20", "20–30", "30–40", "40–50", "50–60", "60–70", "70–80", "80–90", "90–100"]

# Convert to ordered categorical
df['age'] = pd.Categorical(df['age'], categories=age_order, ordered=True)

sns.histplot(
    df[df['readmitted'] == 0]['age'],
    bins=20,
    ax=axes[0],
    stat='density',
    color='steelblue'
)
axes[0].set_title("Not Readmitted")
axes[0].set_xlabel("age")

sns.histplot(
    df[df['readmitted'] == 1]['age'],
    bins=20,
    ax=axes[1],
    stat='density',
    color='darkorange'
)
axes[1].set_title("Readmitted")
axes[1].set_xlabel("age")

positions = range(len(clean_labels))

axes[0].set_xticks(positions)
axes[0].set_xticklabels(clean_labels)

axes[1].set_xticks(positions)
axes[1].set_xticklabels(clean_labels)

plt.suptitle("Distribution of Age by Readmission Outcome")
plt.tight_layout()
plt.show()

## Distribution of Length of Stay

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

sns.histplot(
    df[df['readmitted'] == 0]['time_in_hospital'],
    ax=axes[0],
    bins=14,
    stat='density',
    color='steelblue'
)
axes[0].set_title("Not Readmitted")
axes[0].set_xlabel("time_in_hospital")

sns.histplot(
    df[df['readmitted'] == 1]['time_in_hospital'],
    ax=axes[1],
    bins=14,
    stat='density',
    color='darkorange'
)
axes[1].set_title("Readmitted")
axes[1].set_xlabel("time_in_hospital")

plt.suptitle("Distribution of Time in Hospital by Readmission Outcome")
plt.tight_layout()
plt.show()

## Categorical Cardinality

In [ ]:
cat_cols = df.select_dtypes(include='object').columns          # Identify categorical columns (object dtype)
cardinality = (
    df[cat_cols]
    .nunique()
    .sort_values(ascending=False)
    .to_frame("unique_values")
)
cardinality.style.background_gradient(cmap="Blues")

The cardinality of the diagnostic codes is going to need some attention.

Everything else looks very manageable.

Let's look at some of the categorical fields with mid-range cardinality.

In [ ]:
df['medical_specialty'].unique().tolist()

In [ ]:
df['discharge_disposition_id'].unique().tolist()

In [ ]:
str(df['payer_code'].unique().tolist())

`medical_specialty` is the only attribute with any real semantic complexity, but it’s still small enough for encoding.

## High level correlation overview ✈️

In [ ]:
numeric_cols = list(df.select_dtypes(include=['int64', 'float64']).columns)
numeric_cols.remove('rowID')
corr = df[numeric_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(
    corr,
    cmap="Blues",
    annot=False,
    cbar=True,
    square=True,
    linewidths=0.3,
)

plt.title("Correlation Heatmap (Numeric Features)", fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

No notable off-diagonal correlation structure noted

In [ ]:
corr = df[numeric_cols].corr().round(3)
corr

### Examine the diagnostic codes

In [ ]:
str(df['diag_1'].unique().tolist()[:20])

In [ ]:
str(df['diag_2'].unique().tolist()[:20])

In [ ]:
str(df['diag_3'].unique().tolist()[:20])

In [ ]:
df['diag_1_desc'].unique()[:10].tolist()

In [ ]:
df['diag_2_desc'].unique()[:10].tolist()

In [ ]:
df['diag_3_desc'].unique()[:10].tolist()

In [ ]:
mask = ( (df['diag_1'] == '723') | (df['diag_2'] == '723') | (df['diag_3'] == '723') )
df_723 = df[mask][['diag_1','diag_1_desc','diag_2','diag_2_desc','diag_3','diag_3_desc']]
df_723

It looks like there is a 1:1 relationship between the diagnostic codes and their descriptions. But we should check.

In [ ]:
pairs = pd.concat([
    df[['diag_1', 'diag_1_desc']].rename(columns={'diag_1':'code', 'diag_1_desc':'desc'}),
    df[['diag_2', 'diag_2_desc']].rename(columns={'diag_2':'code', 'diag_2_desc':'desc'}),
    df[['diag_3', 'diag_3_desc']].rename(columns={'diag_3':'code', 'diag_3_desc':'desc'})
], axis=0).dropna()

In [ ]:
pairs

In [ ]:
desc_counts = pairs.groupby('code')['desc'].nunique()
desc_counts

In [ ]:
(desc_counts.min(),desc_counts.max())

This shows a **one-to-one correspondence** (mathematically, a bijection) between diagnosis codes and their textual descriptions

Let's build this as a map.

I performed a Google search on "786: Respiratory abnormality, unspecified"

https://www.aapc.com/codes/icd9-codes/786.00

Home> Codes > ICD-9 Codes > Symptoms, Signs, And Ill-defined Conditions > Symptoms> (786.00)

Respiratory abnormality, unspecified (786.00)

ICD-9 code 786.00 for Respiratory abnormality, unspecified is a medical classification as listed by WHO under the range -SYMPTOMS (780-789)

### Build mappings from code to desc, and vice versa

In [ ]:
code_to_desc = pairs.drop_duplicates('code').set_index('code')['desc'].to_dict()

desc_to_code = pairs.drop_duplicates('desc').set_index('desc')['code'].to_dict()

### Check

In [ ]:
code_to_desc['786']

In [ ]:
desc_to_code['Respiratory abnormality, unspecified']

### Can we get by with using numeric codes (not str) ?

In [ ]:
cols = ['diag_1', 'diag_2', 'diag_3']
non_numeric=[]

for col in cols:
    bad = pd.to_numeric(df[col], errors='coerce').isna() & df[col].notna()
    if bad.any():
        print(f"{col} has non-numeric values:")
        vals = list(df.loc[bad, col].unique())
        non_numeric.append(vals)
        print(vals, "\n")
    else:
        print(f"{col} ✓ all values cast cleanly to float\n")

non_numeric = list(set(x for sub in non_numeric for x in sub))
print(f"Complete list of non-numeric ICD-9 codes:\n{non_numeric}")

### Let's try to work out how to handle these. Start with the "?" symbol.

In [ ]:
df[df['diag_1']=='?'][['diag_1','diag_1_desc']].drop_duplicates()

In [ ]:
df[df['diag_2']=='?'][['diag_2','diag_2_desc']].drop_duplicates()

In [ ]:
df[df['diag_3']=='?'][['diag_3','diag_3_desc']].drop_duplicates()

In [ ]:
known_non_numeric = [x for x in non_numeric if x !='?']
(len(non_numeric), len(known_non_numeric))

In [ ]:
# Wider notebook canvas
display(HTML("<style>.container { width:100% !important; }</style>"))

df_non_numeric_codes = pd.DataFrame([(x,code_to_desc[x]) for x in known_non_numeric],columns=['diag_code','diag_desc'])
df_non_numeric_codes.set_index('diag_code', inplace=True)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', 2000,
                       'display.max_colwidth', None):
    display(df_non_numeric_codes.head(25))

### Having seen the non-numeric codes, now look at the numeric codes with a fractional component

In [ ]:
diag_cols = ['diag_1', 'diag_2', 'diag_3']

fractional = pd.Series(dtype=str)

for col in diag_cols:
    fractional = pd.concat([
        fractional,
        df[col][df[col].astype(str).str.contains(r'^\d+\.\d+$', na=False)]
    ])

fractional_list = sorted(fractional.unique().astype(float).tolist())
print(fractional_list)

In [ ]:
y = [[int(x)] + [x] for x in fractional.unique().astype(float)]
fractional_plus_parent = sorted(set(itertools.chain.from_iterable(y)))
print(fractional_plus_parent)

In [ ]:
df_fractional_codes = pd.DataFrame([(x,code_to_desc[str(x)]) for x in fractional_plus_parent],columns=['diag_code','diag_desc'])
df_fractional_codes.set_index('diag_code', inplace=True)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', 2000,
                       'display.max_colwidth', None):
    display(df_fractional_codes.head(90))

Surely we can collapse these all into the parent code 250 without loss of predictive power?